In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re

In [2]:
df = pd.read_csv('../data/bayut/ready_flats.csv', converters={'amenities': literal_eval})

In [3]:
df.head()

,URL,listing_id,type,building,district,neighborhood,price,beds,baths,surface,...,lat,long,highlights,condition,purpose,desc,completion,furnishing,amenities,added_on
0,https://www.bayut.com/property/details-5825610...,5825610,apartments,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,[1],815.0,...,25.177226,55.308851,Vacant Now | Spacious Layout | 1 Bedroom,any,Buy,Vacant Now | Spacious Layout | 1 Bedroom- Refe...,Ready,NaN,[Balcony or Terrace],"March 18, 2022"
1,https://www.bayut.com/property/details-5825610...,5825610,apartments,Wilton Terraces 2,Mohammed Bin Rashid City,Sobha Hartland,1483773,1.0,[1],815.0,...,25.177226,55.308851,Vacant Now | Spacious Layout | 1 Bedroom,any,Buy,Vacant Now | Spacious Layout | 1 Bedroom- Refe...,Ready,NaN,[Balcony or Terrace],"March 18, 2022"
2,https://www.bayut.com/property/details-5933491...,5933491,apartments,NaN,Jumeirah Village Circle (JVC),Eaton Place,1250000,2.0,[2],1141.0,...,25.060612,55.200228,Kitchen Appliances | Pool View | Best Investment,any,Buy,Azco Real Estate Brokers (LLC) is delighted to...,Ready,Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...","May 8, 2022"
3,https://www.bayut.com/property/details-5933545...,5933545,apartments,Binghatti Mirage,Jumeirah Village Circle (JVC),JVC District 10,650000,1.0,[2],618.0,...,25.064736,55.207565,Brand New | Good Investment | Vacant And Read...,any,Buy,AZCO Real Estate Brokers (LLC) is proud to off...,Ready,Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...","May 8, 2022"
4,https://www.bayut.com/property/details-5933677...,5933677,apartments,NaN,Jumeirah Village Circle (JVC),Rigel Apartments,740000,1.0,[2],933.0,...,25.063390,55.202050,Unfurnished 1BHK + Study Room | Private Garden...,any,Buy,Home 4 Ever Real Estate Brokers is lovely to o...,Ready,Unfurnished,"[Balcony or Terrace, Parking Spaces: 1, Swimmi...","May 8, 2022"


In [4]:
views = ['view', 'views', 'seaview', 'overlooking', 'facing']
luxury = ['luxury', 'luxurious', 'luxuriously', 'premium']
discounted = ['reduced price', 'priced reduced', 'price reduction', 'discounted price', 'negotiable', \
              'price drop', 'price dropped']
cheap = ['affordable', 'cheap', 'cheapest', 'well-priced', 'well priced', 'economical', 'best price']
distressed = ['distress', 'distressed']
upgraded = ['upgraded', 'partially upgraded', 'fully upgraded', 'semi-upgraded', 'semi upgraded', 'renovated']
condition = ['superb condition', 'brand new', 'new building', 'high quality',\
     'well maintained', 'well-maintained', 'high-end finishing', 'excellent condition', 'great condition',\
     'pristine condition', 'immaculate condition','amazing condition', 'mint condition', 'bes condition', \
     'best condition', 'good condition', 'new condition', 'perfect condition', 'tip-top condition', \
     'turnkey condition', 'top condition', 'impeccable condition','mint condiition']
investment = ['investment', 'investor deal', 'investors deal', 'roi', 'investor visa', 'high returns']
tenanted = ['tenanted', 'rented', 'tenant']
vacant = ['vacant']
metro = ['close to metro', 'next to metro', 'near metro']
furnished = ['furnished', 'unfurnished', 'semi furnished', 'semi-furnished']
studio = ['studio']

In [5]:
def get_years(df):
    completion_years = []
    
    for lists in df['amenities']:
        str = ','.join(lists)
        m = re.search('Completion Year: (\d{4})', str)
        if m:
            completion_years.append(m.group(1))
        else:
            completion_years.append(0)
            
    df['completion_year'] = completion_years
    
    
def get_floors(df):
    floors = []
    
    for lists in df['amenities']:
        str = ','.join(lists)
        m = re.search('Floor: (\d+)', str)
        if m:
            floors.append(m.group(1))
        else:
            floors.append('')
            
    df['floor'] = floors
    

def get_features(df, f_list, col_name, col_list):
    for row in df['highlights'].str.lower():
        if any(x in row for x in f_list):
            col_list.append(1)
        else:
            col_list.append(0)
    
    if len(col_list) == len(df['highlights']):
        df[col_name] = col_list
        
        
def get_upgraded(df):
    upgraded_p = []
    
    for row in df['highlights'].str.lower():
        row = row.split(', ')
        y = ''
        for x in upgraded:
            if x in row:
                y = x
                break

        upgraded_p.append(y)
        
    if len(upgraded_p) == len(df['highlights']):
        df['upgraded'] = upgraded_p


def get_luxury(df):
    luxury_col = []
    
    for row in df['highlights'].str.lower():
        row = row.split(', ')
        y = 0
        for x in condition:
            if x in row:
                y = 1
                break

        luxury_col.append(y)
        
    if len(luxury_col) == len(df['highlights']):
        df['luxury'] = luxury_col

In [6]:
def clean_data(df):
    df = df.drop_duplicates(subset='listing_id', keep='first')
    df['baths'] = df['baths'].apply(lambda x: x.replace('[','').replace(']','').replace("'",''))
    df['highlights'] = df['highlights'].apply(lambda x: x.replace('|',',').replace(' ,',','))
    df[['baths', 'beds']] = df[['baths', 'beds']].apply(pd.to_numeric, errors='coerce')
#     df.set_index('listing_id', inplace=True)
    
    df = df[(df['surface'] <= 4000)]
    df = df[df['beds'].notna()]
    
    global df_ready
    df_ready = df[(df['completion'] == 'Ready')]

    df_ready = df_ready[['listing_id', 'URL', 'building', 'district', 'neighborhood', 'price', 'beds', 'baths', \
              'surface', 'lat', 'long', 'highlights', 'furnishing', 'amenities']]
    df_ready = df_ready[~df_ready.index.duplicated(keep='first')]    
 
    get_years(df_ready)
    get_floors(df_ready)
    
    get_features(df_ready, views, 'views', [])
    get_features(df_ready, discounted, 'discounted', [])
    get_features(df_ready, cheap, 'cheap', [])
    get_features(df_ready, distressed, 'distressed', [])
    get_features(df_ready, investment, 'investment', [])
    get_features(df_ready, tenanted, 'tenanted', [])
    get_features(df_ready, vacant, 'vacant', [])
    get_features(df_ready, metro, 'metro', [])
    get_features(df_ready, furnished, 'furnished', [])
    get_features(df_ready, condition, 'condition', [])
    
    get_upgraded(df_ready)
    get_luxury(df_ready)
    
    return df_ready

In [7]:
def get_valuation(df_ready): 
    df_ready['price_sqf'] = df_ready['price'] / df_ready['surface']

    df2 = (df_ready.groupby(['district','beds','baths'])['price_sqf']
             .agg([('median_sqf','median')])
             .reset_index())

    df_ready = df_ready.merge(df2, on=['district', 'beds', 'baths'], how='left')

    diff_percent = ((df_ready['price_sqf'] / df_ready['median_sqf']) * 100) - 100
    df_ready['diff_percent'] = diff_percent

    def val_conditions(row):
        if (row['diff_percent'] <= -50):
            return 'great value'
        elif (row['diff_percent'] > -50) & (row['diff_percent'] <= -30):
            return 'good value'
        elif (row['diff_percent'] > -30) & (row['diff_percent'] <= 30):
            return 'fair value'
        elif (row['diff_percent'] > 30) & (row['diff_percent'] <= 50):
            return 'overvalued'
        elif (row['diff_percent'] > 50):
            return 'highly overvalued' 

    df_ready['valuation'] = df_ready.apply(lambda row: val_conditions(row), axis=1)
    return df_ready

In [8]:
df_ready = clean_data(df)
df_ready = get_valuation(df_ready)
print(df_ready.info())


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_4568\913104631.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['baths'] = df['baths'].apply(lambda x: x.replace('[','').replace(']','').replace("'",''))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_4568\913104631.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['highlights'] = df['highlights'].apply(lambda x: x.replace('|',',').replace(' ,',','))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_4568\913104631.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25353 entries, 0 to 25352
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   listing_id       25353 non-null  int64  
 1   URL              25353 non-null  object 
 2   building         14466 non-null  object 
 3   district         25353 non-null  object 
 4   neighborhood     25062 non-null  object 
 5   price            25353 non-null  int64  
 6   beds             25353 non-null  float64
 7   baths            25353 non-null  int64  
 8   surface          25353 non-null  float64
 9   lat              25353 non-null  float64
 10  long             25353 non-null  float64
 11  highlights       25353 non-null  object 
 12  furnishing       14570 non-null  object 
 13  amenities        25353 non-null  object 
 14  completion_year  25353 non-null  object 
 15  floor            25353 non-null  object 
 16  views            25353 non-null  int64  
 17  discounted  